In [ ]:
import sys
sys.path.append('..')

In [ ]:
from src.database.database_extract import RequestLocalAPI
extract = RequestLocalAPI()

#### Extracting examples based on rowID

In [ ]:
example = extract.getExample(rowId=1)
example

#### Extracting all the examples in the DemoDatabase

In [ ]:
example_all = extract.getAllExamples()
example_all

#### Converting a row from the data from DemoCoreLiteDB into a core variant class

In [ ]:
example_cvc = extract.db_to_cvc(rowId=1)
example_cvc